In [191]:
# importing the librtaries
import numpy as np
import nltk
import underthesea
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer 
import heapq

from __future__ import unicode_literals
import spacy
from spacy.language import Language
import gzip
import shutil
from gensim.models import Word2Vec, KeyedVectors
import os

In [192]:
# text from wikipedia about Elon Musk
txt = """1. Hướng dẫn sử dụng nhà vệ sinh:

- Tư thế ngồi: ngồi hẳn xuống bệ ngồi. Không được đứng trên thành bồn hoặc cho chân lên ngồm xổm.

- Sau khi đi vệ sinh xong, bạn cần xả nước cho sạch. Nếu đi đại tiện, bạn có thể đậy nắp rồi mới nhấn nút xả, tránh nước bẩn văng vào người.

- Các loại giấy rác cần vứt vào thùng rác.

- Ngoài ra, hãy để vòi xịt vào đúng vị trí ban đầu.

2. Hướng dẫn sử dụng bồn rửa tay:

- Bước 1: Chà xát hai hai bàn tay với xà phòng (hoặc nước rửa tay) để sát khuẩn

- Bước 2: Bật nước và rửa tay dưới dòng nước sạch.

- Bước 3: Tắt vòi nước và kiểm tra kĩ, tránh trường hợp nước chảy nhỏ giọt.

- Lưu ý:

+ Bạn không được vứt rác, tóc, giấy vào bồn rửa tay, tránh trường hợp tắc ống.

+ Không được tác động mạnh vào bồn.

3. Hướng dẫn sử dụng máy sấy tay:

- Bước 1: Đưa tay vào dưới máy sấy, úp tay, duỗi thẳng các ngón tay. Khi có gió (khí) thổi xuống, bạn đưa tay của mình ra vào từ từ, nhiều lần.

- Bước 2: Ngửa lòng bàn tay lên và giữa nguyên thao tác như bước 1.

- Bước 3: Xoa hai tay với nhau để tay khô hẳn.

Lưu ý: máy sấy sẽ tự động tắt khi hoạt động liên tục trong 60 giây. Nếu muốn tiếp tục sử dụng, bạn hãy đưa tay ra khỏi thiết bị và làm lại lần lượt 3 bước trên."""

In [193]:
nltk.sent_tokenize(txt)

['1.',
 'Hướng dẫn sử dụng nhà vệ sinh:\n\n- Tư thế ngồi: ngồi hẳn xuống bệ ngồi.',
 'Không được đứng trên thành bồn hoặc cho chân lên ngồm xổm.',
 '- Sau khi đi vệ sinh xong, bạn cần xả nước cho sạch.',
 'Nếu đi đại tiện, bạn có thể đậy nắp rồi mới nhấn nút xả, tránh nước bẩn văng vào người.',
 '- Các loại giấy rác cần vứt vào thùng rác.',
 '- Ngoài ra, hãy để vòi xịt vào đúng vị trí ban đầu.',
 '2.',
 'Hướng dẫn sử dụng bồn rửa tay:\n\n- Bước 1: Chà xát hai hai bàn tay với xà phòng (hoặc nước rửa tay) để sát khuẩn\n\n- Bước 2: Bật nước và rửa tay dưới dòng nước sạch.',
 '- Bước 3: Tắt vòi nước và kiểm tra kĩ, tránh trường hợp nước chảy nhỏ giọt.',
 '- Lưu ý:\n\n+ Bạn không được vứt rác, tóc, giấy vào bồn rửa tay, tránh trường hợp tắc ống.',
 '+ Không được tác động mạnh vào bồn.',
 '3.',
 'Hướng dẫn sử dụng máy sấy tay:\n\n- Bước 1: Đưa tay vào dưới máy sấy, úp tay, duỗi thẳng các ngón tay.',
 'Khi có gió (khí) thổi xuống, bạn đưa tay của mình ra vào từ từ, nhiều lần.',
 '- Bước 2: Ng

In [194]:
def read_stopwords(filepath):
    lines = []

    # Open the file for reading
    with open(filepath, 'r', encoding='utf-8') as file:
        # Read each line in the file
        for line in file:
            # Append each line to the list, removing leading and trailing whitespace
            lines.append(line.strip())
            
    return lines

In [195]:
vi_stopwords = read_stopwords("./vietnamese-stopwords.txt")

In [196]:
#class for preprocessing and creating word embedding

class Preprocessing:
    #constructor
    def __init__(self,txt):
        # Tokenization
        nltk.download('punkt')  #punkt is nltk tokenizer 
        # breaking text to sentences
        tokens = underthesea.sent_tokenize(txt) 
        self.tokens = tokens
        self.tfidfvectoriser=TfidfVectorizer()
        self.word2vect_model = self.initWord2Vect()

    # Data Cleaning
    # remove extra spaces
    # convert sentences to lower case 
    # remove stopword
    def clean_sentence(self, sentence, stopwords=False):
        sentence = sentence.lower().strip()
        sentence = re.sub(r'[^a-zA-Z0-9\sÀÁẢẠÃĂẰẮẲẶẴÂẦẤẨẬẪÈÉẺẸẼÊỀẾỂỆỄÌÍỈỊĨÒÓỎỌÕÔỒỐỔỘỖƠỜỚỞỢỠÙÚỦỤŨƯỪỨỬỰỮỲÝỶỴỸàáảạãăằắẳặẵâầấẩậẫèéẻẹẽêềếểệễìíỉịĩòóỏọõôồốổộỗơờớởợỡùúủụũưừứửựữỳýỷỵỹđ]+', '', sentence)
        if stopwords:
            sentence = remove_stopwords(sentence)
        return sentence

    # store cleaned sentences to cleaned_sentences
    def get_cleaned_sentences(self,tokens, stopwords=False):
        cleaned_sentences = []
        for line in tokens:
            cleaned = self.clean_sentence(line, stopwords)
            cleaned_sentences.append(cleaned)
        return cleaned_sentences

    #do all the cleaning
    def cleanall(self):
        cleaned_sentences = self.get_cleaned_sentences(self.tokens, stopwords=True)
        cleaned_sentences_with_stopwords = self.get_cleaned_sentences(self.tokens, stopwords=False)
        return [cleaned_sentences,cleaned_sentences_with_stopwords]

    def initWord2Vect(self):
        # Get it from https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.vi.vec
        vectors_gzip = "./weights/wiki.vi.model.bin.gz"
        vectors_loc="./weights/wiki.vi.model.bin"
        if not os.path.exists(vectors_loc):
            with gzip.open(vectors_gzip, 'rb') as f_in:
                with open(vectors_loc, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

        model = KeyedVectors.load_word2vec_format(vectors_loc, binary=True)
        
        return model

    # TF-IDF Vectorizer
    def TFIDF(self,cleaned_sentences):
        self.tfidfvectoriser.fit(cleaned_sentences)
        tfidf_vectors=self.tfidfvectoriser.transform(cleaned_sentences)
        return tfidf_vectors

    #tfidf for question
    def TFIDF_Q(self,question_to_be_cleaned):
        tfidf_vectors=self.tfidfvectoriser.transform([question_to_be_cleaned])
        return tfidf_vectors

    # main call function
    def doall(self):
        cleaned_sentences, cleaned_sentences_with_stopwords = self.cleanall()
        tfidf = self.TFIDF(cleaned_sentences)
        word2vect = self.Word2Vect(cleaned_sentences)
        return [cleaned_sentences,cleaned_sentences_with_stopwords,tfidf, word2vect]
    
    # Get word2vect
    def Word2Vect(self, cleaned_sentences):
        result = []
        for sentence in cleaned_sentences:
            sentence = [self.getWord2Vect(token) for token in list(gensim.utils.tokenize(sentence))]
            result.append(np.mean(np.array(sentence), axis=0))
        return result
    
    def getWord2Vect(self, word):
        try:
            vect = self.word2vect_model[word]
        except:
            vect = np.random.uniform(-0.25, 0.25, 400)
        return vect

    def sentenceWord2Vect(self, sentence):
        word2vect_sen = [self.getWord2Vect(token) for token in list(gensim.utils.tokenize(sentence))]
        return np.mean(np.array(word2vect_sen), axis=0)


In [197]:
#class for answering the question.
class AnswerMe:
    #cosine similarity
    def __init__(self, type_embedding="word2vect"):
        self.type_embedding = type_embedding
    
    def Cosine(self, question_vector, sentence_vector):
        dot_product = np.dot(question_vector, sentence_vector.T)
        denominator = (np.linalg.norm(question_vector) * np.linalg.norm(sentence_vector))
        return dot_product/denominator
    
    #Euclidean distance
    def Euclidean(self, question_vector, sentence_vector):
        vec1 = question_vector.copy()
        vec2 = sentence_vector.copy()
        if self.type_embedding != "word2vect":
            if len(vec1)<len(vec2): vec1,vec2 = vec2,vec1
            vec2 = np.resize(vec2,(vec1.shape[0],vec1.shape[1]))
        return np.linalg.norm(vec1-vec2)

    # main call function
    def answer(self, question_vector, sentence_vector, method):
        if method==1: return self.Euclidean(question_vector,sentence_vector)
        else: return self.Cosine(question_vector,sentence_vector)


In [198]:
def RetrieveAnswer(question_embedding, tfidf_vectors,method=1, type_embedding=None):
    similarity_heap = []
    if method==1: max_similarity = float('inf')
    else: max_similarity = -1
    index_similarity = -1

    for index, embedding in enumerate(tfidf_vectors):  
        find_similarity = AnswerMe(type_embedding)
        print(type(embedding))
        if not isinstance(question_embedding, np.ndarray):
            question_embedding = (question_embedding).toarray()
        if not isinstance(embedding, np.ndarray):
            embedding = (embedding).toarray()
        similarity = find_similarity.answer(question_embedding,embedding , method).mean()
        if method==1:
            heapq.heappush(similarity_heap,(similarity,index))
        else:
            heapq.heappush(similarity_heap,(-similarity,index))
            
    return similarity_heap



In [199]:
# Put Your question here
user_question = "Cho tôi hướng dẫn sử dụng máy sấy tay"
#define method
method = 1

In [200]:
preprocess = Preprocessing(txt)
cleaned_sentences,cleaned_sentences_with_stopwords,tfidf_vectors, word2vect_dict = preprocess.doall()

question = preprocess.clean_sentence(user_question, stopwords=True)
question_embedding = preprocess.TFIDF_Q(question)
question_word2vect = preprocess.sentenceWord2Vect(question)

similarity_heap = RetrieveAnswer(question_embedding , tfidf_vectors ,method, "tfidf")

# Using word2vect
# similarity_heap = RetrieveAnswer(question_word2vect, word2vect_dict ,method )

[nltk_data] Downloading package punkt to /home/tuannm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>


/home/tuannm/anaconda3/envs/common/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/tuannm/anaconda3/envs/common/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [201]:
print("Question: ", user_question)

# number of relevant solutions you want here it will print 2
number_of_sentences_to_print = 2

while number_of_sentences_to_print>0 and len(similarity_heap)>0:
    x = similarity_heap.pop(0)
    print(cleaned_sentences_with_stopwords[x[1]])
    number_of_sentences_to_print-=1

Question:  Cho tôi hướng dẫn sử dụng máy sấy tay
hướng dẫn sử dụng máy sấy tay

 bước 1 đưa tay vào dưới máy sấy úp tay duỗi thẳng các ngón tay
2
